In [1]:
#Necessary Packages
import numpy as np
import copy
from scipy.signal import argrelextrema

class FHMC_Analysis:
    def __init__(self,lnZ0,V,beta,Debug=False,Extrapolated=False):
        self.N = np.array([])
        self.lnPi = np.array([])
        self.energy = np.array([])
        self.energy2 = np.array([])
        self.lnZ0 = lnZ0
        self.V = V
        self.beta = beta
        self.Debug = Debug
        self.Extrapolated = Extrapolated
    def reweight(self,lnZ_rw, Normalize=False, Output=False, CheckTail=False):
        # Reweighting operation
        lnPi_temp = self.lnPi + self.N*(lnZ_rw-self.lnZ0)
        # Shift relative to max value
        lnPi_temp = lnPi_temp - max(lnPi_temp)
        # Normalize, if desired
        if Normalize: lnPi_temp = lnPi_temp - np.log( np.sum( np.exp(lnPi_temp) ) )
        # Create a child object and replace the lnPi and lnZ
        child = copy.deepcopy(self) # double ensure that the child is a new object, not a pointer
        child.lnZ0 = lnZ_rw
        child.lnPi = np.array(lnPi_temp)
        if CheckTail:
            #minima = argrelextrema( lnPi_temp, np.less, order=10 )
            #HAVE TO DEAL WITH NO MINIMA CASE -> take this into a separate function
            maxima = argrelextrema( lnPi_temp, np.greater, order=10 )
            delta = np.abs(lnPi_temp[int(maxima[-1])] - lnPi_temp[-1])
            if delta < 10.:
                raise Exception('\nReweighted to lnZ = '+str(lnZ_rw)+'\n' \
                                'Insufficient Tail: '+str(delta)+'\n' \
                                'Maxima at: '+str(int(maxima[-1]))+'  '+str( lnPi_temp[int(maxima[-1])] )+'\n'\
                                'Tail   at: '+str(self.N[-1])+'  '+str( lnPi_temp[-1] ) )
        return child
    def temperature_extrapolate(self,beta_extrap, order=1, Normalize=False, Output=False, CheckTail=False):
        mu = self.lnZ0/self.beta
        if order < 1:
            raise Exception('\nExtrapolation Order = '+str(order)+' not possible\n' \
                                'Check your input data')
        #First Order Extrapolation
        lnPi_temp = self.lnPi + (beta_extrap - self.beta)*(mu*self.N - self.energy)
        #NOTE: Inconsequential terms were skipped; they are all independent of N
        #      and so do not affect the relative values of Pi
        if order == 2:
            #Second Order Extrapolation            
            lnPi_temp = lnPi_temp + 0.50 * ((beta_extrap - self.beta)**2)*(self.energy2 - (self.energy)**2)
            #NOTE: As for first order extrap, inconsequential terms were skipped
        if order > 2:
            raise Exception('\nExtrapolation Order = '+str(order)+' not possible\n' \
                                'Check your input data')
        if Normalize: lnPi_temp = lnPi_temp - np.log( np.sum( np.exp(lnPi_temp) ) )
        if CheckTail:
            maxima = argrelextrema( lnPi_temp, np.greater, order=10 )
            delta = np.abs(lnPi_temp[int(maxima[-1])] - lnPi_temp[-1])
            if delta < 10.:
                raise Exception('\nExtrapolated to beta = '+str(beta_extrap)+'\n' \
                                'Insufficient Tail: '+str(delta)+'\n' \
                                'Maxima at: '+str(int(maxima[-1]))+'  '+str( lnPi_temp[int(maxima[-1])] )+'\n'\
                                'Tail   at: '+str(self.N[-1])+'  '+str( lnPi_temp[-1] ) )
        #QUESTION: SHOULD THIS FUNCTION RETURN A NEW OBJECT?
        # ADVANTAGES: child object can be used just like the parent
        # DISADVANTAGES: what about the energy / higher moments?
        #    (we lose the highest order moment while extrapolation)
        # TO DO: Must recalculate lnZ after extrapolation [mu or mu_star stays fixed, not lnZ]
        # ADVICE: DO NOT APPLY EXTRAPOLATION METHOD TO EXTRAPOLATED LNPI -> BAD IDEA!!!!
        #  Perhaps include a class descriptor that states whether the lnPi is 'pristine' or not
        return lnPi_temp

#Function for computing ensemble properties
def ensemble_properties(self):
    Pi = np.exp( self.lnPi - max( self.lnPi ) )
    denom = np.sum( Pi )
    Navg = np.dot( self.N, Pi )/denom
    N2avg = np.dot( (self.N)**2, Pi )/denom
    Uavg = np.dot( self.energy, Pi )/denom
    U2avg = np.dot( self.energy2, Pi )/denom
    UNavg = np.dot( np.multiply(self.N,self.energy), Pi )/denom
    #Determine a better way to output all of these moments
    return Navg,N2avg,Uavg,U2avg,UNavg
    
#Function for computing phase properties
def phase_properties(self):
    # Phase-specific properties
    maxima = argrelextrema( self.lnPi, np.greater, order=10 )
    if len(maxima) == 1:
        Nbound = self.N[-1]
        phases = 1
    elif ln(maxima) == 2:
        minima = argrelextrema( self.lnPi, np.less, order=10 )
        Nbound = int(minima[0])
        phases = 2
    else:
        raise Exception('\nMore than two maxima in lnPi\n'+\
                        'Current lnZ = '+str(self.lnZ))
    #Create lists for properties computation
    Navg = [ [0. for x in range(phases)] for i in range(2) ]
    Uavg = [0. for x in range(phases)]
    UNavg = [0. for x in range(phases)]
    for i in range(phases):
        #NOTE: the "numpy.multiply" function is the "Hadamard" Product,
        # e.g., element-wise multiplication of two tensors of the same rank;
        # it yields a tensor of the same rank as the input tensors
        if i == 0:
            #  Shift lnPi to prevent round-off truncation in lnPi -> Pi conversion
            Pi = np.exp( self.lnPi - max( self.lnPi[:Nbound] ) )
            denom = np.sum( Pi[:Nbound] )
            for j in range(2):
                Navg[j][i] = np.dot( self.N[:Nbound]**(j+1), Pi[:Nbound] )/denom
            Uavg[i] = np.dot( self.energy[:Nbound], Pi[:Nbound] )/denom
            UNavg[i] = np.dot( np.multiply(self.N[:Nbound],self.energy[:Nbound]),Pi[:Nbound] )/denom
        elif i == 1:
            #  Shift lnPi to prevent round-off truncation in lnPi -> Pi conversion
            Pi = np.exp( self.lnPi - max( self.lnPi[Nbound+1:]) )
            denom = np.sum( Pi[Nbound+1:] )
            for j in range(2):
                Navg[i] = np.dot( self.N[Nbound+1:]**(j+1), Pi[Nbound+1:] )/denom
            Uavg[i] = np.dot( self.energy[Nbound+1:], Pi[Nbound+1:] )/denom
            UNavg[i] = np.dot( np.multiply(self.N[Nbound+1:],self.energy[Nbound+1:]),Pi[Nbound+1:] )/denom            
    return Navg,Uavg,UNavg


#Bind Custom Functions to Class
FHMC_Analysis.phase_properties = phase_properties
FHMC_Analysis.ensemble_properties = ensemble_properties

In [2]:
beta = 1./0.730
root_dir = '/mnt/WS_PN107151/dsideriu/Research/Monte_Carlo/LJuVT_WL_TMMC/PROD_2.5sigma/'

In [3]:
# Confined Fluid information
conf_location = root_dir+'Ar_MWCNT1043_R5.9786_T_073/'
conf_input_file = 'setup_runs.json'
conf_runs = ['r1']

N = []
lnPi = []
energy = []

lnpi_input = conf_location+'r1/ljsf.t073.rcyl_5.97860s.l_12s.r1.lnpi.dat'
with open(lnpi_input,mode='r') as f:
    text_in = f.readlines()
for line in text_in:
    N.append(int(float(line.split()[0])))
    lnPi.append(float(line.split()[1]))

energy_input = conf_location+'r1/ljsf.t073.rcyl_5.97860s.l_12s.r1.energy.dat'
with open(lnpi_input,mode='r') as f:
    text_in = f.readlines()
for line in text_in:
    energy.append(float(line.split()[1]))

lnZ_conf  = -2.6306135*beta

# Instantiate the Confined Fluid object
confined = FHMC_Analysis(lnZ_conf,1.,beta)
confined.N = np.array(N)
confined.lnPi = np.array(lnPi)
confined.energy = np.array(energy)
#confined.energy2 = energy2

In [4]:
# Instantiate the Bulk Fluid object
bulk_location = root_dir+'Ar_Bulk_V512_T_073/'
bulk_input_file = 'setup_runs.json'
bulk_runs = ['r1']

N = []
lnPi = []
energy = []

lnpi_input = bulk_location+'r1/ljsf.t073.bulk.v512.r1.lnpi.dat'
with open(lnpi_input,mode='r') as f:
    text_in = f.readlines()
for line in text_in:
    N.append(int(float(line.split()[0])))
    lnPi.append(float(line.split()[1]))

energy_input = conf_location+'r1/ljsf.t073.bulk.v512.r1.energy.dat'
with open(lnpi_input,mode='r') as f:
    text_in = f.readlines()
for line in text_in:
    energy.append(float(line.split()[1]))

lnZ_bulk  = -2.890*beta
V_bulk = 512.

# Instantiate the Confined Fluid object
bulk = FHMC_Analysis(lnZ_bulk,V_bulk,beta)
bulk.N = np.array(N)
bulk.lnPi = np.array(lnPi)
bulk.energy = np.array(energy)

In [14]:
lnZ_min = beta*(-10.)
lnZ_max = beta*(-2.8883204153999515)

isotherm_points = 20
for lnZ_rw in np.linspace(lnZ_min,lnZ_max,isotherm_points):
    #print(lnZ_rw)
    new = confined.reweight(lnZ_rw=lnZ_rw,Normalize=True,CheckTail=True)
    props = new.phase_properties()
    print(lnZ_rw/beta, props[0][0][0])


-10.0 76.81528651638111
-9.625701074494735 147.05293962291375
-9.25140214898947 201.82131237411755
-8.877103223484204 232.6828882670468
-8.502804297978937 251.47990679846095
-8.128505372473672 265.1103781685142
-7.754206446968405 275.59553067570965
-7.37990752146314 284.47461859561196
-7.005608595957874 292.4678629588298
-6.631309670452608 300.37289229499453
-6.257010744947343 309.2373498988897
-5.882711819442077 320.77542009908933
-5.508412893936811 342.9128410800815
-5.134113968431545 380.5984390402518
-4.759815042926279 439.04371392593225
-4.385516117421014 510.81027598823755
-4.0112171919157475 575.6705208970966
-3.6369182664104818 702.7538751861257
-3.2626193409052155 919.4092549678392
-2.8883204153999515 950.266170872178


In [23]:
# Isotherm Package

def isotherm_and_Qst(conf,bulk,lnZ_min,lnZ_max,points):
    lnZ = np.linspace(lnZ_min,lnZ_max,points)
    pressure = []
    isotherm = []
    if bulk.beta != conf.beta:
        raise ValueError('ERROR: temperatures do not match!')
    beta = conf.beta
    #V_acc = conf.Vacc
    for lnZ_RW in lnZ:
        bulk_child = bulk.reweight(lnZ_rw=lnZ_RW,CheckTail=False,Normalize=True)
        pressure.append(-bulk_child.lnPi[0]/beta/bulk.V) ### QUICK HACK
        confined_child = confined.reweight(lnZ_rw=lnZ_RW,Normalize=True,CheckTail=True)
        props = confined_child.phase_properties()
        isotherm.append(props[0][0][0])
    return pressure,isotherm


isotherm_points = 20
lnZ_min = beta*(-10.)
lnZ_max = beta*(-2.8883204153999515)  #This is the saturation chemical potential

pressure, isotherm = isotherm_and_Qst(confined,bulk,lnZ_min,lnZ_max,isotherm_points)
#fig = plt.figure()
#plt.plot(pressure,isotherm)
#plt.xlabel(r'$p*$')
#plt.ylabel(r'$<N>/V_{total}$')
#plt.show()
#Qst_isostere = []

for i in range(isotherm_points):
    print(pressure[i],isotherm[i])

8.204329156037764e-07 76.81528651638111
1.3700122505675178e-06 147.05293962291375
2.2877425616614457e-06 201.82131237411755
3.820252500544254e-06 232.6828882670468
6.379412499050211e-06 251.47990679846095
1.065308715083792e-05 265.1103781685142
1.7790190338878577e-05 275.59553067570965
2.9710020299658454e-05 284.47461859561196
4.9619696954014724e-05 292.4678629588298
8.288070147326415e-05 300.37289229499453
0.00013846285382698297 309.2373498988897
0.00023139184557883743 320.77542009908933
0.0003868915544601005 342.9128410800815
0.0006474583212415004 380.5984390402518
0.0010851270510007045 439.04371392593225
0.0018232895296278818 510.81027598823755
0.003077219869299394 575.6705208970966
0.00523513777492459 702.7538751861257
0.00904296067629977 919.4092549678392
0.017027584082646825 950.266170872178


In [5]:
lnZ_min = beta*(-10.)
lnZ_max = beta*(-2.8883204153999515)  #This is the saturation chemical potential

for lnZ_RW in np.linspace(lnZ_min, lnZ_max, 20):
    bulk_child = bulk.reweight(lnZ_rw=lnZ_RW,CheckTail=True,Normalize=True)
    print(lnZ_RW, bulk_child.lnPi[0])

(array([], dtype=int64),)


TypeError: only size-1 arrays can be converted to Python scalars